In [1]:
# Copyright (c) 2024, InfinityQ Technology Inc.

# Prompt the user and wait for an answer before installing dependencies.
# Note: Usage of a virtual Python environment is recommended.
if input("Install Python dependencies? [y/n]").lower() == "y":
    !pip install -r "requirements.txt" --upgrade
else:
    print("Skipping dependency installation.")

import numpy as np
from problem_gen import *
from titanq import Model, Vtype, Target, S3Storage
import logging

## Setting Credentials for TitanQ
The user should configure their TitanQ API key here. For very large problems, the user must also configure an AWS Access key, AWS Secret Access key and AWS Bucket Name.

In [2]:
logging.getLogger('botocore').setLevel(logging.CRITICAL)
logging.getLogger('urllib3').setLevel(logging.CRITICAL)

# Enter your API key here
# Obtain your API key by contacting --> support@infinityq.tech
# Example: TITANQ_DEV_API_KEY = "00000000-0000-0000-0000-000000000000"
TITANQ_DEV_API_KEY = None

# Specify AWS keys and bucket name for solving very large problems
# AWS_ACCESS_KEY = "Access Key"
# AWS_SECRET_ACCESS_KEY = "Secret Access Key"
# AWS_BUCKET_NAME = "Bucket Name"

## Problem Definition
Load input matrices:
- Flow Matrix = Indicates the dependencies between materials

- Distance Matrix = Lists the distance between locations

- Production Cost Matrix = Lists the costs for a facility to finish a material

All matrices are stored as Float32 numpy arrays.

Example of a Flow Matrix with 4 materials:

$$\begin{bmatrix} 
 & material_1 & material_2 & material_3 & material_4 \\ 
material_1 & 0 & 1 & 1 & 0 \\ 
material_2 & 1 & 0 & 0 & 1 \\
material_3 & 1 & 0 & 0 & 1 \\
material_4 & 0 & 1 & 1 & 0
\end{bmatrix}$$

The Flow Matrix describes the dependencies between the processing of materials.

In [3]:
flow_matrix = np.genfromtxt("input/flow_matrix.csv", delimiter=",")

Example of a Distance Matrix with 4 locations:

$$\begin{bmatrix} 
 & location_1 & location_2 & location_3 & location_4 \\ 
location_1 & \lambda & 0 & 5 & 8 \\ 
location_2 & 0 & \lambda & 5 & 8 \\
location_3 & 5 & 0 & \lambda & 6 \\
location_4 & 8 & 8 & 6 & \lambda
\end{bmatrix}$$
where $\lambda$ is a parameter that can be used to tune how much the materials are spread out across the locations.

The Distance Matrix contains information regarding the distance between locations.

In [4]:
distance_matrix = np.genfromtxt("input/distance_matrix.csv", delimiter=",")

Example of a Production Cost Matrix with 4 facilities and 4 materials:

$$\begin{bmatrix} 
 & facility_1 & facility_2 & facility_3 & facility_4 \\ 
material_1 & 10 & 0 & 5 & 0 \\ 
material_2 & 0 & 12 & 0 & 10 \\
material_3 & 10 & 0 & 5 & 0 \\
material_4 & 0 & 12 & 0 & 10
\end{bmatrix}$$
where every cell with the value 0 can be replaced by a penalty factor to penalize invalid assignments.

The Production Cost Matrix contains information regarding the processing costs associated with a certain material and facility.

In [5]:
production_cost_matrix = np.genfromtxt("input/production_cost_matrix.csv", delimiter=",")

## Generate Inputs for the TitanQ SDK

In [6]:
weights = generate_weights(flow_matrix, distance_matrix)

bias = generate_bias(production_cost_matrix)

In [7]:
# Initialize the model
model = Model(
    api_key=TITANQ_DEV_API_KEY,
    # Insert storage_client parameter and specify corresponding AWS keys and bucket name for solving very large problems
    # storage_client=S3Storage(
    #     access_key=AWS_ACCESS_KEY,
    #     secret_key=AWS_SECRET_ACCESS_KEY,
    #     bucket_name=AWS_BUCKET_NAME
    # )
)

# Construct the problem
x = model.add_variable_vector('x', len(weights), Vtype.BINARY)
model.set_objective_matrices(weights, bias, Target.MINIMIZE)

#Adding constraints using expressions
for i in range(len(distance_matrix)):
    expr = sum(x[4*i:4*i+4]) == 1
    model.add_constraint_from_expression(expr)

## Call TitanQ Solver

Define the variable name along with the variable type.

Set the weights, bias, weight constraints and bias constraints defined above previously and insert them into the SDK.

Specify whether to minimize or maximize the Hamiltonian as the objective of the solver (Set to Target.MINIMIZE by default).

Additional hyperparameters which can be tuned include:

- beta = Scales the problem by this factor (inverse of temperature). A lower beta allows for easier escape from local minima, while a higher beta is more likely to respect penalties and constraints.

- coupling_mult = Strength of the minor embedding for the TitanQ specific hardware.

- timeout_in_secs = Maximum runtime of the solver in seconds.

- num_chains = Number of parallel runs executed by the solver. A larger number of parallel runs generally leads to higher quality solutions.


In [8]:
# Set hyperparameters and call the solver
num_chains = 128

Tmin = 0.1
Tmax = 1000
beta = (1/np.linspace(Tmin, Tmax, num_chains, dtype=np.float32)).tolist()
response = model.optimize(beta=beta, timeout_in_secs=20, coupling_mult=50, num_engines=1, num_chains=num_chains)

# Print the solution and keep track of state of lowest Ising energy
print("-" * 15, "+", "-" * 26, sep="")
print("Ising energy   | Result vector")
print("-" * 15, "+", "-" * 26, sep="")
ctr = 0
for ising_energy, result_vector in response.result_items():
    print(f"{ising_energy: <14f} | {result_vector}")
    if ctr == 0:
        lowest_ising_energy = ising_energy
        index = 0
    elif ising_energy < lowest_ising_energy:
        lowest_ising_energy = ising_energy
        index = ctr
    ctr += 1

---------------+--------------------------
Ising energy   | Result vector
---------------+--------------------------
25.000000      | [0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0.]


## Transforming TitanQ Output into Assignment

The solution returned by TitanQ is transformed into a matrix with the rows corresponding to materials and the columns corresponding to facilities. The solution is stored in an Excel file. The format of the dictionary is: assignment[material] = facility.

In [9]:
result_matrix = reshape_result_vector(response.result_vector()[index], len(flow_matrix), len(distance_matrix))

material_names = ["Material_1", "Material_2", "Material_3", "Material_4"]
facility_names = ["Facility_1", "Facility_2", "Facility_3", "Facility_4"]

assignment = generate_assignment(result_matrix, material_names, facility_names)
save_assignment(result_matrix,material_names,facility_names)

print(assignment)

{'Material_1': 'Facility_2', 'Material_2': 'Facility_3', 'Material_3': 'Facility_3', 'Material_4': 'Facility_1'}
